# *Task* - Processing of Traffic Light Status Information in MPC-Planner

## Summary

Identify and implement MPC-planner functionalities to improve trajectory planning at traffic lights.

- [Background and Motivation](#background-and-motivation)
- [Task](#task)
- [Required Tools and Data](#required-tools-and-data)
- [Hints](#hints)

## Background and Motivation

The modules of vehicle guidance within the AD software stack are responsible for enabling a comfortable, efficient, and safe guidance of the vehicle. To this end, the guidance system relies on a model of the environment, derived by several perception-, V2X- and environment-interpretation modules (and potentially additional map-information).

The most important task is to guide the vehicle safely through its environment, e.g., avoiding collisions with other traffic participants in the local environment. Moreover, the vehicle is obliged to comply with traffic regulations, e.g., to stop at a red traffic light.

A popular approach to trajectory planning is *Model Predictive Control (MPC)* including cost functions modeling different constraints. As an example, the ACDC MPC-planner employs the following cost term in order to stop at a *red* traffic light.

$$ j_{tl} = \left( w_{tl} \cdot \left( \cos \left( \frac{\pi \cdot {d_{tl}}^2}{{d_{ref,tl}}^2} \right) + 1 \right)\right)^2 \qquad \textrm{if traffic light is red} $$

with Euclidean distance $d_{tl}$ to the traffic light, reference distance $d_{ref,tl}$, and cost factor $w_{tl}$.

This implementation has several drawbacks:
- no consideration of the stop line geometry, instead the traffic light is treated as a point in space;
- no difference in behavior in front of vs. behind the traffic light reference position;
- no incorporation of future traffic light state changes.

One possible effect of this cost term, if analyzed on its own, could be the vehicle going around the traffic light without consideration of the stop line, since only the Euclidean distance to a reference point is considered.

Below you can observe a traffic light-aware planned trajectory in green, along with the traffic light status change information *(see [ACDC Exercise: SPaT Processing](https://github.com/ika-rwth-aachen/acdc/wiki/Section-5-SPaT-Processing))*.

![](./assets/planning.png)

## Task

The task is to identify and implement MPC-planner functionalities to improve trajectory planning at traffic lights.

### Subtasks

> ***Note:*** *The subtasks listed below do not have to be followed strictly. They serve the purpose of guiding you along your own research for this topic.*

1. Research literature regarding the modeling of traffic lights within the OCP (Optimal Control Problem) in the context of automated driving.
2. Understand ACDC Course's planner implementation using the [Control Toolbox](https://github.com/ethz-adrl/control-toolbox).
3. Identify and implement reasonable extensions to the MPC-planner functions and cost terms to improve trajectory planning at traffic lights (see some suggestions in *Hints*).
4. Evaluate the resulting driving behavior:
   1. define performance metrics to evaluate your cost function improvements (e.g., stop distance to traffic light);
   2. perform simulations using your implementation and gather data as rosbags;
   3. evaluate and discuss the results (e.g., by analyzing the rosbag data in Python using the [rosbag API](http://wiki.ros.org/ROS/Tutorials/reading%20msgs%20from%20a%20bag%20file)).
5. Document your research, developed approach, and evaluations in a Jupyter notebook report. Explain and reproduce individual parts of your implemented functions with exemplary data.


## Required Tools and Data

### Tools

- ROS
- [ACDC Motion Planning](https://github.com/ika-rwth-aachen/acdc/tree/main/catkin_workspace/src/workshops/section_4)
- [Control Toolbox](https://github.com/ethz-adrl/control-toolbox)

### Data

- closed-loop simulation *(from ACDC Course; use the [vehicle_guidance_v2x.launch-File](https://github.com/ika-rwth-aachen/acdc/blob/main/catkin_workspace/src/workshops/section_4/trajectory_planner/launch/vehicle_guidance_v2x.launch))*

## Hints

### Relevant ACDC Sections

- **Vehicle Guidance**
  - Vehicle Guidance on Guidance Level
- **Connected Driving**
  - V2I-Communication

### Ideas for Possible Improvements

> ***Note:*** *The suggestions detailed below do not have to be chosen as for implementation. They only serve as inspiration.*

- consideration of stop line geometry instead of only keeping a distance to a specific traffic light reference point
- differentiation between approaching the stop line and behavior after the stop line has (accidentally) been crossed
- integration of traffic light status changes based on V2X communication, e.g. slowing down even if the current state is *green* or accelerating to cross the traffic light before turning *red*
- ...

### Stop Line Geometry

The internal traffic light representation of the ACDC planner contains a so called *[ingress lane](https://github.com/ika-rwth-aachen/acdc/blob/main/catkin_workspace/src/workshops/section_4/trajectory_planner/include/trajectory_planner.hpp#L312)* as a vector of points. The last point within this list should represent the position of the stop-line. To derive the dimensions of this line you could assume a constant width and generate 
the boundary points as an orthogonal projection with respect to the ingress lane.

### State-Change Timing Information

The SPAT Visualization indicates the remaining time until the actual traffic light state changes. Using the *[vehicle_guidance_v2x.launch-File](https://github.com/ika-rwth-aachen/acdc/blob/main/catkin_workspace/src/workshops/section_4/trajectory_planner/launch/vehicle_guidance_v2x.launch)* the RVIZ output should look like the image above. You may get inspired by the [given implementation of the visualization](https://github.com/ika-rwth-aachen/acdc/blob/main/catkin_workspace/src/workshops/section_5/etsi_visualization/src/SPATViz.cpp#L165) to derive the *Time-to-Change* of a traffic light and integrate this into the trajectory planner.